In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
def load_data():
    """데이터 로드 함수"""
    train = pd.read_csv('data/train.csv')
    test = pd.read_csv('data/test.csv')
    return train, test

def preprocess_data(train_df, test_df):
    """데이터 전처리 함수"""
    # 제거할 컬럼 목록
    columns_to_drop = ['ID', 'PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일']
    
    # 학습 데이터 전처리
    train_processed = train_df.copy()
    train_processed.drop(columns=columns_to_drop, inplace=True)
    train_processed.fillna(0, inplace=True)
    
    # 범주형 변수 인코딩
    categorical_cols = train_processed.select_dtypes(include=["object"]).columns.tolist()
    label_encoders = {}
    
    for col in categorical_cols:
        le = LabelEncoder()
        train_processed[col] = le.fit_transform(train_processed[col].astype(str))
        label_encoders[col] = le
    
    # 수치형 변수 정규화
    target_col = "임신 성공 여부"
    numeric_cols = [col for col in train_processed.select_dtypes(include=["int64", "float64"]).columns 
                   if col != target_col]
    
    scaler = MinMaxScaler()
    train_processed[numeric_cols] = scaler.fit_transform(train_processed[numeric_cols])
    
    # 테스트 데이터 전처리
    test_processed = test_df.copy()
    test_processed.drop(columns=columns_to_drop, inplace=True)
    test_processed.fillna(0, inplace=True)
    
    # 범주형 변수 동일하게 인코딩 적용
    for col in categorical_cols:
        if col in test_processed.columns and col in label_encoders:
            # 학습 데이터에서 학습한 인코더 사용
            test_processed[col] = test_processed[col].astype(str)
            test_processed[col] = label_encoders[col].transform(test_processed[col])
    
    # 수치형 변수 동일하게 정규화 적용
    test_processed[numeric_cols] = scaler.transform(test_processed[numeric_cols])
    
    return train_processed, test_processed, label_encoders, scaler

if __name__ == "__main__":
    # 데이터 로드
    train_df, test_df = load_data()
    
    # 데이터 전처리
    train_processed, test_processed, label_encoders, scaler = preprocess_data(train_df, test_df)
    
    # 결과 확인
    print(f"학습 데이터 형태: {train_processed.shape}")
    print(f"테스트 데이터 형태: {test_processed.shape}")